This notebook gives a demonstration of how you might quickly use this library to visualise clusters.

## Data

Let's start by importing a dataset. In particular we're going to toy around with a subset from a [customer service dataset from Twitter](https://www.kaggle.com/thoughtvector/customer-support-on-twitter).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

df = pd.read_csv("../data/tesco_support.csv").loc[lambda d: ~d['text'].str.contains("https")]
texts = list(df['text'].sample(2000))
texts[:3]

["@Tesco what's wrong with the car park at Tesco in Hanley? Traffic not moved for half an hournow",
 '@Tesco is the policy of chaining your wheelchairs together and putting the coin slot UNDER the seat going to be rolled out nationwide? #disabiltyawareness',
 '@Tesco I found a rat wandering around in your store.']

We're going to pretend like we might want to predict labels in this data. What might be some good labels? 

## Arrays 

Let's try a embed-reduce-then-visualise technique for this. 

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

pipe = make_pipeline(TfidfVectorizer(), TruncatedSVD())

X = pipe.fit_transform(texts)

## Visualize

Given that we now have a numeric representation `X`, we can visualise and explore. 

Hint; you can click + drag in the embedding space! 

In [4]:
from cluestar import plot_text

plot_text(X, texts)

alt.HConcatChart(...)

In [5]:
import pathlib 

schema = plot_text(X, texts).to_json()
# pathlib.Path("../docs/plot_one.json").write_text(schema)

## Improve 

You might be able to unravel some clusters, but we might improve the experience by using UMAP to reduce the embeddings down. UMAP typically keeps the local clusters more intact. More info can be found [here](https://pair-code.github.io/understanding-umap/). 

In [6]:
from umap import UMAP

pipe = make_pipeline(TfidfVectorizer(), UMAP())

X = pipe.fit_transform(texts)

plot_text(X, texts)

/home/vincent/Development/cluestar/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-03-25 15:56:13.004633: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-25 15:56:13.004652: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


alt.HConcatChart(...)

You might be able to add color to certain clusters by highlighting moments when a specific word is used.

In [7]:
import pathlib 

schema = plot_text(X, texts).to_json()
# pathlib.Path("../docs/plot_two.json").write_text(schema)

In [8]:
plot_text(X, texts, color_words=["plastic", "voucher", "deliver"])

alt.HConcatChart(...)

In [9]:
import pathlib 

schema = plot_text(X, texts, color_words=["plastic", "voucher", "deliver"]).to_json()
# pathlib.Path("../docs/plot_three.json").write_text(schema)

## More Fancy 

You can try to get extra fancy by introducing language models. The [whatlies](https://github.com/koaning/whatlies/) library makes it easy to produce a scikit-learn pipeline for that.

In [10]:
from whatlies.language import UniversalSentenceLanguage

pipe = make_pipeline(
    UniversalSentenceLanguage(), 
    UMAP()
)

X = pipe.fit_transform(texts)

plot_text(X, texts, color_words=["plastic", "voucher", "deliver"])

2022-03-25 15:56:21.273819: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-25 15:56:21.273955: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-25 15:56:21.273963: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-25 15:56:21.273974: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-03-25 15:56:21.274103: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

alt.HConcatChart(...)

The goal here is exploration. By using these embedding tricks we're able to confirm that many folks complain about deliveries and vouchers. These could be interesting labels to try and predict, so a next step might be to attach labels to this dataset via something like [prodigy](https://prodi.gy/).

In [11]:
import pathlib 

schema = plot_text(X, texts, color_words=["plastic", "voucher", "deliver"]).to_json()
# pathlib.Path("../docs/plot_four.json").write_text(schema)

## Visualising Predictions

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    UniversalSentenceLanguage(), 
    UMAP()
)

pred_pipe = make_pipeline(
    CountVectorizer(), 
    LogisticRegression(class_weight="balanced")
)

In [36]:
subset = df.sample(2000).assign(label=lambda d: d['text'].str.contains("delivery|late|driver"))
texts, y = list(subset['text']), subset['label']

In [37]:
X = pipe.fit_transform(texts)
probas = pred_pipe.fit(texts, y).predict_proba(texts)

In [38]:
plot_text(X, texts, color_array=probas[:, 1])

alt.HConcatChart(...)

In [40]:
schema = plot_text(X, texts, color_array=probas[:, 1]).to_json()
pathlib.Path("../docs/plot_five.json").write_text(schema)

859888

## Version Numbers

This notebook was made with the following tool versions:

In [16]:
%load_ext watermark
%watermark --machine --python --packages numpy,pandas,scikit-learn,whatlies,tensorflow,umap

Python implementation: CPython
Python version       : 3.8.6
IPython version      : 8.1.1

numpy       : 1.19.5
pandas      : 1.4.1
scikit-learn: 1.0.2
whatlies    : 0.6.5
tensorflow  : 2.4.4
umap        : 0.5.2

Compiler    : GCC 10.2.0
OS          : Linux
Release     : 5.11.0-7614-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

